In [ ]:
# first four FRD

import os
import pandas as pd
import csv


FirstRateData = os.listdir("/data/workspace_files/FirstRateData/")
FirstRateData.remove("readme.txt")
FirstRateData.remove("B6_EW.zip")
FirstRateData.remove("SI_YM.zip")
FirstRateData.remove("GC_HG.zip")
FirstRateData.remove("LBS_SDA.zip")
FirstRateData.remove("ZC_ZW.zip")
FirstRateData.remove("A6_AD_B0_B6.zip")

while len(FirstRateData) > 0:
    current_file = []
    current_file.append(FirstRateData[0])
    FirstRateData.remove(current_file[0])
    for i in range(len(FirstRateData)-1,-1,-1):
        if FirstRateData[i][0:3] == current_file[0][0:3]:
            current_file.append(FirstRateData[i])
            FirstRateData.remove(FirstRateData[i])
    
    print(current_file)
    # #TODO
    # if len(current_file) < 3:
    #     continue
    
    current_file = sorted(current_file)

    frames = []
    for i in range(0, len(current_file)):
        data_tmp = pd.read_csv('/data/workspace_files/FirstRateData/' + current_file[i], sep=",", header=None)
        data_tmp.columns = ["DateTime", "Open", "High", "Low", "Close", "Volume"]
        data_tmp = data_tmp.set_index('DateTime')
        data_tmp.index = pd.to_datetime(data_tmp.index)
        frames.append(data_tmp)

    data = pd.concat(frames)

    # print(data)

    hour_prior_volume = []
    hour_prior_range_high = []
    hour_prior_range_low = []
    overnight_range_high = []
    overnight_range_low = []
    overnight_volume = []

    new_data = []

    new_data.append(["DateTime", "hour_prior_volume", "hour_prior_range", "overnight_range", "overnight_volume"])
    year = -1
    month = -1
    day = -1
    prev_index = -1
    for index, row in data.iterrows():
        if year == -1:
            year = index.year
            month = index.month
            day = index.day
            prev_index = index
        if index.year == year and index.month == month and index.day == day:
            
            if (index.hour == 5 and index.minute >= 30) or (index.hour == 6 and index.minute <= 30):
                hour_prior_volume.append(row['Volume'])
            
            if (index.hour == 8 and index.minute >= 30) or (index.hour == 9 and index.minute <= 30):
                hour_prior_range_high.append(row['High'])
                hour_prior_range_low.append(row['Low'])

            if (index.hour >= 16):
                overnight_range_high.append(row['High'])
                overnight_range_low.append(row['Low'])
                overnight_volume.append(row['Volume'])
        else:
            #TODO
            if (index.hour <= 9) or (index.hour == 9 and index.minute <= 30):
                overnight_range_high.append(row['High'])
                overnight_range_low.append(row['Low']) 
                overnight_volume.append(row['Volume'])

            if len(hour_prior_volume) != 0 or len(hour_prior_range_high)!= 0 or len(overnight_range_high) !=0 or len(overnight_volume) != 0:
                new_data.append([str(prev_index.year)+"-"+str(prev_index.month)+"-"+str(prev_index.day),
                             sum(hour_prior_volume),
                             (max(hour_prior_range_high) - min(hour_prior_range_low)) if len(hour_prior_range_high) != 0 else -1,
                             (max(overnight_range_high) - min(overnight_range_low)) if len(overnight_range_high) != 0 else -1,
                             sum(overnight_volume)])
            
            hour_prior_volume = []
            hour_prior_range_high = []
            hour_prior_range_low = []
            overnight_range_high = []
            overnight_range_low = []
            overnight_volume = []
            
            year = index.year
            month = index.month
            day = index.day
            prev_index = index


    str_tmp = str(current_file[0][0:3])
    
    split_string = str_tmp.split(".", 1)
    split_string = split_string[0].split("_", 1)
    
    str_tmp = split_string[0]
    print(str_tmp)
    with open("FirstRateData/" + str_tmp + ".csv", "w+") as my_csv:
        csvWriter = csv.writer(my_csv,delimiter=',')
        csvWriter.writerows(new_data)

In [ ]:
# first four kibot

import os
import pandas as pd
import csv


# FirstRateData = os.listdir("/data/workspace_files/Kibot/")

FirstRateData = ["JY.txt", "US.txt", "NG.txt", "NE.txt", "M6E.txt", "NIY.txt", "QM.txt", "PX.txt", "HO.txt", "NKD.txt", "MGC.txt",
"M6A.txt", "M6B.txt", "JE.txt", "MJY.txt", "RP.txt", "RY.txt", "MCD.txt", "RF.txt", "BR.txt", "RU.txt", "RA.txt", "XAE.txt", "XAF.txt",
"XAU.txt", "XAV.txt", "XAY.txt", "XAK.txt", "XAI.txt", "MSF.txt", "XAP.txt", "XAB.txt", "SEK.txt", "AC.txt", "NOK.txt", "AJY.txt",
"PJY.txt", "EAD.txt", "ECD.txt"]

while len(FirstRateData) > 0:
    current_file = []
    current_file.append(FirstRateData[0])
    FirstRateData.remove(current_file[0])
    for i in range(len(FirstRateData)-1,-1,-1):
        if FirstRateData[i][0:3] == current_file[0][0:3]:
            current_file.append(FirstRateData[i])
            FirstRateData.remove(FirstRateData[i])
    
    print(current_file)
    
    current_file = sorted(current_file)

    frames = []
    for i in range(0, len(current_file)):

        data_tmp = pd.read_csv('/data/workspace_files/Kibot/' + current_file[i], sep=",", header=None)
        data_tmp.columns = ["year", "time", "Open", "High", "Low", "Close", "Volume"]
        data_tmp["DateTime"] = data_tmp["year"] + data_tmp["time"]
        data_tmp = data_tmp.drop(['year', 'time'], axis=1)
        
        ll = data_tmp.values.tolist()

        for i3 in range(len(ll)):
            strr = ll[i3][5]
            mm = strr[0:2]
            dd = strr[3:5]
            yy = strr[6:10]
            tt = strr[10:]
            ff_strr = yy + "-" + mm + "-" + dd + " " + tt
            ll[i3][5] = ff_strr
        
        from pandas import DataFrame

        data_tmp = DataFrame(ll,columns=["Open", "High", "Low", "Close", "Volume", "DateTime"])
        data_tmp = data_tmp.set_index('DateTime')
        data_tmp.index = pd.to_datetime(data_tmp.index)
        frames.append(data_tmp)

    data = pd.concat(frames)

    # print(data)

    hour_prior_volume = []
    hour_prior_range_high = []
    hour_prior_range_low = []
    overnight_range_high = []
    overnight_range_low = []
    overnight_volume = []

    new_data = []

    new_data.append(["DateTime", "hour_prior_volume", "hour_prior_range", "overnight_range", "overnight_volume"])
    year = -1
    month = -1
    day = -1
    prev_index = -1
    for index, row in data.iterrows():
        if year == -1:
            year = index.year
            month = index.month
            day = index.day
            prev_index = index
        if index.year == year and index.month == month and index.day == day:
            
            if (index.hour == 5 and index.minute >= 30) or (index.hour == 6 and index.minute <= 30):
                hour_prior_volume.append(row['Volume'])
            
            if (index.hour == 8 and index.minute >= 30) or (index.hour == 9 and index.minute <= 30):
                hour_prior_range_high.append(row['High'])
                hour_prior_range_low.append(row['Low'])

            if (index.hour >= 16):
                overnight_range_high.append(row['High'])
                overnight_range_low.append(row['Low'])
                overnight_volume.append(row['Volume'])
        else:
            #TODO
            if (index.hour <= 9) or (index.hour == 9 and index.minute <= 30):
                overnight_range_high.append(row['High'])
                overnight_range_low.append(row['Low']) 
                overnight_volume.append(row['Volume'])

            if len(hour_prior_volume) != 0 or len(hour_prior_range_high)!= 0 or len(overnight_range_high) !=0 or len(overnight_volume) != 0:
                new_data.append([str(prev_index.year)+"-"+str(prev_index.month)+"-"+str(prev_index.day),
                             sum(hour_prior_volume),
                             (max(hour_prior_range_high) - min(hour_prior_range_low)) if len(hour_prior_range_high) != 0 else -1,
                             (max(overnight_range_high) - min(overnight_range_low)) if len(overnight_range_high) != 0 else -1,
                             sum(overnight_volume)])
            
            hour_prior_volume = []
            hour_prior_range_high = []
            hour_prior_range_low = []
            overnight_range_high = []
            overnight_range_low = []
            overnight_volume = []
            
            year = index.year
            month = index.month
            day = index.day
            prev_index = index


    str_tmp = str(current_file[0][0:3])
    print("kibot/" + str_tmp + ".csv")

    split_string = str_tmp.split(".", 1)

    str_tmp = split_string[0]

    with open("kibot/" + str_tmp + ".csv", "w+") as my_csv:
        csvWriter = csv.writer(my_csv,delimiter=',')
        csvWriter.writerows(new_data)
        print("kibot/" + str_tmp + ".csv")

In [ ]:
# rest FRD

import math
import os
import pandas as pd
import csv


tmptmp = 0
while tmptmp < 1:
    tmptmp = 1

    current_file = ["ZT_2000_2009.txt", "ZT_2010_2019.txt", "ZT_2020_2020.txt"]

    frames = []
    for i in range(0, len(current_file)):
        data_tmp = pd.read_csv('/data/workspace_files/FirstRateData/' + current_file[i], sep=",", header=None)
        data_tmp.columns = ["DateTime", "Open_2", "High_2", "Low_2", "Close_2", "Volume_2"]
        data_tmp = data_tmp.set_index('DateTime')
        data_tmp.index = pd.to_datetime(data_tmp.index)
        frames.append(data_tmp)

    data_2 = pd.concat(frames)

    current_file = ["ZF_2000_2009.txt", "ZF_2010_2019.txt", "ZF_2020_2020.txt"]

    frames = []
    for i in range(0, len(current_file)):
        data_tmp = pd.read_csv('/data/workspace_files/FirstRateData/' + current_file[i], sep=",", header=None)
        data_tmp.columns = ["DateTime", "Open_5", "High_5", "Low_5", "Close_5", "Volume_5"]
        data_tmp = data_tmp.set_index('DateTime')
        data_tmp.index = pd.to_datetime(data_tmp.index)
        frames.append(data_tmp)

    data_5 = pd.concat(frames)

    current_file = ["ZN_2000_2009.txt", "ZN_2010_2019.txt", "ZN_2020_2020.txt"]

    frames = []
    for i in range(0, len(current_file)):
        data_tmp = pd.read_csv('/data/workspace_files/FirstRateData/' + current_file[i], sep=",", header=None)
        data_tmp.columns = ["DateTime", "Open_10", "High_10", "Low_10", "Close_10", "Volume_10"]
        data_tmp = data_tmp.set_index('DateTime')
        data_tmp.index = pd.to_datetime(data_tmp.index)
        frames.append(data_tmp)

    data_10 = pd.concat(frames)

    current_file = ["US.txt"]

    frames = []
    for i in range(0, len(current_file)):
        data_tmp = pd.read_csv('/data/workspace_files/Kibot/' + current_file[i], sep=",", header=None)
        data_tmp.columns = ["year", "time", "Open_30", "High_30", "Low_30", "Close_30", "Volume_30"]
        data_tmp["DateTime"] = data_tmp["year"] + data_tmp["time"]
        data_tmp = data_tmp.drop(['year', 'time'], axis=1)
        
        ll = data_tmp.values.tolist()

        for i3 in range(len(ll)):
            strr = ll[i3][5]
            mm = strr[0:2]
            dd = strr[3:5]
            yy = strr[6:10]
            tt = strr[10:]
            ff_strr = yy + "-" + mm + "-" + dd + " " + tt
            ll[i3][5] = ff_strr
        
        from pandas import DataFrame

        data_tmp = DataFrame(ll,columns=["Open_30", "High_30", "Low_30", "Close_30", "Volume_30", "DateTime"])
        data_tmp = data_tmp.set_index('DateTime')
        data_tmp.index = pd.to_datetime(data_tmp.index)
        frames.append(data_tmp)

    data_30 = pd.concat(frames)

    data = pd.concat([data_2, data_5, data_10, data_30], axis=1)

    print(data)
 
    # data.to_csv("test.csv")
 
    b_10_5_year_overnight = []
    b_10_5_year_hourprior = []
    b_10_5_year_hour_to_open = []
    ten_is_increase_hour = -1
    ten_is_decrease_hour = -1
    ten_is_increase_hour_open = -1
    ten_is_decrease_hour_open = -1

    b_5_2_year_overnight = []
    b_5_2_year_hourprior = []
    b_5_2_year_hour_to_open = []
    five_is_increase_hour = -1
    five_is_decrease_hour = -1
    five_is_increase_hour_open = -1
    five_is_decrease_hour_open = -1

    b_30_10_year_overnight = []
    b_30_10_year_hourprior = []
    b_30_10_year_hour_to_open = []
    thirty_is_increase_hour = -1
    thirty_is_decrease_hour = -1
    thirty_is_increase_hour_open = -1
    thirty_is_decrease_hour_open = -1

    new_data = []

    new_data.append(["DateTime", "10_5_year_overnight", "10_5_year_hourprior", "10_5_year_hour_to_open", "ten_is_increase_hour",
                     "ten_is_decrease_hour", "ten_is_increase_hour_open", "ten_is_decrease_hour_open",
                     "5_2_year_overnight", "5_2_year_hourprior", "5_2_year_hour_to_open", "five_is_increase_hour",
                     "five_is_decrease_hour", "five_is_increase_hour_open", "five_is_decrease_hour_open",
                     "30_10_year_overnight", "30_10_year_hourprior", "30_10_year_hour_to_open",
                     "thirty_is_increase_hour", "thirty_is_decrease_hour", "thirty_is_increase_hour_open",
                     "thirty_is_decrease_hour_open"                
                     ])
    year = -1
    month = -1
    day = -1
    prev_index = -1
    for index, row in data.iterrows():
        if year == -1:
            year = index.year
            month = index.month
            day = index.day
            prev_index = index
        if index.year == year and index.month == month and index.day == day:

            if index.hour >= 16 and len(b_10_5_year_overnight) == 0 and (not math.isnan(row['High_10'])) and (not math.isnan(row['High_5'])):
                b_10_5_year_overnight.append(row['High_10'] - row['High_5'])
            if index.hour == 8 and index.minute == 30 and (not math.isnan(row['High_10'])) and (not math.isnan(row['High_5'])):
                b_10_5_year_hourprior.append(row['High_10'] - row['High_5'])
            if index.hour == 9 and index.minute == 30 and (not math.isnan(row['High_10'])) and (not math.isnan(row['High_5'])):
                b_10_5_year_hour_to_open.append(row['High_10'] - row['High_5'])

            if index.hour >= 16 and len(b_5_2_year_overnight) == 0 and (not math.isnan(row['High_5'])) and (not math.isnan(row['High_2'])):
                b_5_2_year_overnight.append(row['High_5'] - row['High_2'])
            if index.hour == 8 and index.minute == 30 and (not math.isnan(row['High_5'])) and (not math.isnan(row['High_2'])):
                b_5_2_year_hourprior.append(row['High_5'] - row['High_2'])
            if index.hour == 9 and index.minute == 30 and (not math.isnan(row['High_5'])) and (not math.isnan(row['High_2'])):
                b_5_2_year_hour_to_open.append(row['High_5'] - row['High_2'])

            if index.hour >= 16 and len(b_30_10_year_overnight) == 0 and (not math.isnan(row['High_30'])) and (not math.isnan(row['High_10'])):
                b_30_10_year_overnight.append(row['High_30'] - row['High_10'])
            if index.hour == 8 and index.minute == 30 and (not math.isnan(row['High_30'])) and (not math.isnan(row['High_10'])):
                b_30_10_year_hourprior.append(row['High_30'] - row['High_10'])
            if index.hour == 9 and index.minute == 30 and (not math.isnan(row['High_30'])) and (not math.isnan(row['High_10'])):
                b_30_10_year_hour_to_open.append(row['High_30'] - row['High_10'])
            
        else:
            if len(b_10_5_year_overnight) > 0 and len(b_10_5_year_hour_to_open) > 0:
                if b_10_5_year_overnight[0] > b_10_5_year_hour_to_open[0]:
                    ten_is_increase_hour = 1
                else:
                    ten_is_increase_hour = 0    

            if len(b_10_5_year_overnight) > 0 and len(b_10_5_year_hour_to_open) > 0:
                if b_10_5_year_overnight[0] > b_10_5_year_hour_to_open[0]:
                    ten_is_decrease_hour = 0
                else:
                    ten_is_decrease_hour = 1  

            if len(b_10_5_year_hourprior) > 0 and len(b_10_5_year_hour_to_open) > 0:
                if b_10_5_year_hour_to_open[0] > b_10_5_year_hourprior[0]:
                    ten_is_increase_hour_open = 1
                else:
                    ten_is_increase_hour_open = 0  

            if len(b_10_5_year_hourprior) > 0 and len(b_10_5_year_hour_to_open) > 0:
                if b_10_5_year_hour_to_open[0] > b_10_5_year_hourprior[0]:
                    ten_is_decrease_hour_open = 1
                else:
                    ten_is_decrease_hour_open = 0 


            if len(b_5_2_year_overnight) > 0 and len(b_5_2_year_hour_to_open) > 0:
                if b_5_2_year_overnight[0] > b_5_2_year_hour_to_open[0]:
                    five_is_increase_hour = 1
                else:
                    five_is_increase_hour = 0    

            if len(b_5_2_year_overnight) > 0 and len(b_5_2_year_hour_to_open) > 0:
                if b_5_2_year_overnight[0] > b_5_2_year_hour_to_open[0]:
                    five_is_decrease_hour = 0
                else:
                    five_is_decrease_hour = 1  

            if len(b_5_2_year_hourprior) > 0 and len(b_5_2_year_hour_to_open) > 0:
                if b_5_2_year_hour_to_open[0] > b_5_2_year_hourprior[0]:
                    five_is_increase_hour_open = 1
                else:
                    five_is_increase_hour_open = 0  

            if len(b_5_2_year_hourprior) > 0 and len(b_5_2_year_hour_to_open) > 0:
                if b_5_2_year_hour_to_open[0] > b_5_2_year_hourprior[0]:
                    five_is_decrease_hour_open = 1
                else:
                    five_is_decrease_hour_open = 0 


            if len(b_30_10_year_overnight) > 0 and len(b_30_10_year_hour_to_open) > 0:
                if b_30_10_year_overnight[0] > b_30_10_year_hour_to_open[0]:
                    thirty_is_increase_hour = 1
                else:
                    thirty_is_increase_hour = 0    

            if len(b_30_10_year_overnight) > 0 and len(b_30_10_year_hour_to_open) > 0:
                if b_30_10_year_overnight[0] > b_30_10_year_hour_to_open[0]:
                    thirty_is_decrease_hour = 0
                else:
                    thirty_is_decrease_hour = 1  

            if len(b_30_10_year_hourprior) > 0 and len(b_30_10_year_hour_to_open) > 0:
                if b_30_10_year_hour_to_open[0] > b_30_10_year_hourprior[0]:
                    thirty_is_increase_hour_open = 1
                else:
                    thirty_is_increase_hour_open = 0  

            if len(b_30_10_year_hourprior) > 0 and len(b_30_10_year_hour_to_open) > 0:
                if b_30_10_year_hour_to_open[0] > b_30_10_year_hourprior[0]:
                    thirty_is_decrease_hour_open = 1
                else:
                    thirty_is_decrease_hour_open = 0 


            new_data.append([str(prev_index.year) + "-" + str(prev_index.month) + "-" + str(prev_index.day),
                             b_10_5_year_overnight[0] if len(b_10_5_year_overnight) > 0 else -1,
                             b_10_5_year_hourprior[0] if len(b_10_5_year_hourprior) > 0 else -1,
                             b_10_5_year_hour_to_open[0] if len(b_10_5_year_hour_to_open) > 0 else -1,
                             ten_is_increase_hour,
                             ten_is_decrease_hour,
                             ten_is_increase_hour_open,
                             ten_is_decrease_hour_open,
                             b_5_2_year_overnight[0] if len(b_5_2_year_overnight) > 0 else -1,
                             b_5_2_year_hourprior[0] if len(b_5_2_year_hourprior) > 0 else -1,
                             b_5_2_year_hour_to_open[0] if len(b_5_2_year_hour_to_open) > 0 else -1,
                             five_is_increase_hour,
                             five_is_decrease_hour,
                             five_is_increase_hour_open,
                             five_is_decrease_hour_open,
                             b_30_10_year_overnight[0] if len(b_30_10_year_overnight) > 0 else -1,
                             b_30_10_year_hourprior[0] if len(b_30_10_year_hourprior) > 0 else -1,
                             b_30_10_year_hour_to_open[0] if len(b_30_10_year_hour_to_open) > 0 else -1,
                             thirty_is_increase_hour,
                             thirty_is_decrease_hour,
                             thirty_is_increase_hour_open,
                             thirty_is_decrease_hour_open
                             ])
            
            
            year = index.year
            month = index.month
            day = index.day
            prev_index = index


    str_tmp = str(current_file[0][0:3])

    with open("FirstRateData/bounds.csv", "w+") as my_csv:
        csvWriter = csv.writer(my_csv,delimiter=',')
        csvWriter.writerows(new_data)

In [ ]:
import pandas as pd

FRD = ["ES.csv", "NQ.csv", "A6.csv", "B6.csv", "BTC.csv", "CL.csv", "E6.csv", "ED.csv", "GC.csv", "HG.csv", "ME.csv", 
        "MES.csv", "PL.csv", "RTY.csv", "SI.csv", "VX.csv", "YM.csv", "ZC.csv", "ZS.csv", "EW.csv", "ZF.csv", "ZN.csv",
        "ZT.csv", "LBS.csv", "LE.csv", "HE.csv", "ZW.csv", "PA.csv", "E1.csv", "MP.csv", "AD.csv", "UB.csv", "ZQ.csv",
        "ZL.csv", "ZM.csv", "RB.csv", "BZ.csv"]

Kibot = ["JY.csv", "US.csv", "NG.csv", "NE.csv", "M6E.csv", "NIY.csv", "QM.csv", "PX.csv", "HO.csv", "NKD.csv", "MGC.csv",
        "M6A.csv", "M6B.csv", "JE.csv", "MJY.csv", "RP.csv", "RY.csv", "MCD.csv", "RF.csv", "BR.csv", "RU.csv", "RA.csv", "XAE.csv", "XAF.csv",
        "XAU.csv", "XAV.csv", "XAY.csv", "XAK.csv", "XAI.csv", "MSF.csv", "XAP.csv", "XAB.csv", "SEK.csv", "AC.csv", "NOK.csv", "AJY.csv",
        "PJY.csv", "EAD.csv", "ECD.csv"]

frames = []

for i in range(0, len(FRD)):
    split_string = FRD[i].split(".", 1)
    
    data_tmp = pd.read_csv('/data/workspace_files/processed_data/FirstRateData/' + FRD[i], sep=",", header=None)
    data_tmp.columns = ["DateTime", split_string[0]+"_hour_prior_volume", split_string[0]+"_hour_prior_range", 
                        split_string[0]+"_overnight_range", split_string[0]+"_overnight_volume"]
    data_tmp = data_tmp.iloc[1:]
    data_tmp = data_tmp.set_index('DateTime')
    data_tmp.index = pd.to_datetime(data_tmp.index)

    if split_string[0] == "BTC" or split_string[0] == "MES":
        data_tmp = data_tmp.loc[~data_tmp.index.duplicated(keep='first')]
    # print(data_tmp[data_tmp.index.duplicated()])
    frames.append(data_tmp)

for i in range(0, len(Kibot)):
    split_string = Kibot[i].split(".", 1)
    
    data_tmp = pd.read_csv('/data/workspace_files/processed_data/kibot/' + Kibot[i], sep=",", header=None)
    data_tmp.columns = ["DateTime", split_string[0]+"_hour_prior_volume", split_string[0]+"_hour_prior_range", 
                        split_string[0]+"_overnight_range", split_string[0]+"_overnight_volume"]
    data_tmp = data_tmp.iloc[1:]
    data_tmp = data_tmp.set_index('DateTime')
    data_tmp.index = pd.to_datetime(data_tmp.index)
    # print(data_tmp[data_tmp.index.duplicated(keep=False)])
    frames.append(data_tmp)

data = pd.concat(frames, axis=1)

print(data)

data.to_csv('FRD+Kibot.csv')
